<a href="https://colab.research.google.com/github/anmolsabharwal/RedDevils/blob/master/RD_Goals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing Important libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sqlite3
from google.colab import drive
import datetime
import plotly.figure_factory as ff
 
 
drive.mount('/content/drive')
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
gws=pd.read_csv("/content/drive/My Drive/combined_2016-2020-1.csv",encoding="ISO-8859-1")
gws.head(60)

In [ ]:
gws.rename(columns={'ï»¿name':'name'},inplace=True)
gws.columns


In [ ]:
#converting the datatypes of columns
gws=gws.astype({'name':'category','assists':int, 'bonus':int, 'bps':int, 'clean_sheets':int,'creativity':float,'element':int,'goals_conceded':int, 'ict_index':float,
        'influence':float,'minutes':int, 'opponent_team':int, 'own_goals':int,'penalties_missed':int, 'penalties_saved':int,'red_cards':int, 'round':int, 'saves':int, 'selected':int,
        'threat':float,'was_home':'bool','total_points':int, 'transfers_balance':int, 'transfers_in':int, 'transfers_out':int,'value':int, 'yellow_cards':int,'gw':int})

In [ ]:
# cheking for null values
sns.set_style('dark')
sns.heatmap(gws.isnull(),yticklabels=False,cbar=False,cmap='inferno')

In [ ]:
gws = gws.assign(id=(gws['name']).astype('category').cat.codes)

In [ ]:
gws= gws[gws['minutes'] > 0] 


In [ ]:
# a function to decide the fixture difficulty ratio based on the position of the opponent team
def fd(x):
  x=int(x)
  if x<=6:
    return '3'
  elif x>6 and x<=14:
    return '1'
  else:
    return '2'

In [ ]:
gws.rename(columns={'opponent_team':'fdr'},inplace=True)


In [ ]:
def goal(x):
  x=int(x)
  if x==0:
    return 1
  elif x>=1 and x<=2:
    return 2
  else:
    return 3

In [ ]:
gws['goals']=gws['goals_scored'].apply(goal).astype(float)
gws.head(50)

In [ ]:
# setting up the 'was_home' column to create binary attributes
def was_home(a):
  if a==False:
    #print('Away')
    return str('Away')
  else:
    #print('Home')
    return str('Home')

In [ ]:
gws['home']=gws['was_home'].apply(was_home)

In [ ]:
gws.drop(['element','fixture','round','transfers_balance','transfers_in','transfers_out','team_a_score','was_home',	'team_h_score','round'],axis=1, inplace=True)

In [ ]:
gws['value']=gws['value']*0.1
gws1=gws.copy()
gws1.head()

In [ ]:
gws1['Last_Week_Goals'] = gws1.groupby(['id'])['goals_scored'].shift()
gws1['Last_2_Week_Goals'] = gws1.groupby(['id'])['goals_scored'].shift(2)
gws1['Last_3_Week_Goals'] = gws1.groupby(['id'])['goals_scored'].shift(3)

In [ ]:
window=5
ipm =((gws1.groupby('id',as_index=False)['influence'].expanding().sum()/gws1.groupby('id',as_index=False)['minutes'].expanding().sum())*90).astype(float)
gws1["Influence_90"] = ipm.reset_index(level=0, drop=True)

tpm =((gws1.groupby('id',as_index=False)['threat'].expanding().sum()/gws1.groupby('id',as_index=False)['minutes'].expanding().sum())*90).astype(float)
gws1["Threat_90"] = ipm.reset_index(level=0, drop=True)

cpm =((gws1.groupby('id',as_index=False)['creativity'].expanding().sum()/gws1.groupby('id',as_index=False)['minutes'].expanding().sum())*90).astype(float)
gws1["Creativity_90"] = ipm.reset_index(level=0, drop=True)

gpm =((gws1.groupby('id',as_index=False)['goals_scored'].expanding().sum()/gws1.groupby('id',as_index=False)['minutes'].expanding().sum())*90).astype(float)
gws1["Goals_90"] = gpm.reset_index(level=0, drop=True)

ict =gws1.groupby('id',as_index=False)['ict_index'].rolling(window=window).sum().shift(1).astype(float)
gws1["ICT Form"] = ict.reset_index(level=0, drop=True)

In [ ]:
def f(df):
    df['Goalstreak'] = (df['goals_scored'] > 0).cumsum()
    df['cumsum'] = np.nan
    df.loc[df['goals_scored'] == 0, 'cumsum'] = df['Goalstreak']
    df['cumsum'] = df['cumsum'].fillna(method='ffill')
    df['cumsum'] = df['cumsum'].fillna(0)
    df['Goal Streak'] = df['Goalstreak'] - df['cumsum']
    df.drop(['Goalstreak', 'cumsum'], axis=1, inplace=True)
    df['Goal Streak']=df['Goal Streak'].shift(1)
    return df

In [ ]:
gws1=gws1.groupby('id').apply(f)


In [ ]:
df=gws1.groupby('id',as_index=False).apply(lambda group: group.iloc[window:,])
X1=df.reset_index(level=0,drop=True)
X1.to_csv('/content/drive/My Drive/train.csv')
X1.sort_values('gw')

In [ ]:
X1.columns

In [ ]:
X=X1[['Last_Week_Goals', 'Last_2_Week_Goals', 'Last_3_Week_Goals', 'Goals_90',
       'ICT Form', 'Influence_90', 'Threat_90', 'Creativity_90',
       'Goal Streak']]
y=X1['goals']

In [ ]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn.preprocessing import StandardScaler,MaxAbsScaler,MinMaxScaler
from imblearn.over_sampling import ADASYN

scaler=MinMaxScaler(feature_range=(0,1))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=153)
oversample = SMOTE(sampling_strategy='all')
X_train,y_train=oversample.fit_resample(X_train,y_train)
# print(sum(y_train==0))
# print(sum(y_train==1))
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
ovr=LogisticRegression(max_iter=200,penalty='l2')
ovr.fit(X_train,y_train)

In [ ]:
predict=ovr.predict(X_test)
predictp=ovr.predict_proba(X_test)

In [ ]:
df1=pd.DataFrame({'Actual':y_test,'Predicted':predict.round()})
# a=df1[0]=predictp[:,0].round(decimals=2)
# b=df1[1]=predictp[:,1].round(decimals=2)
df1.head(50)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix,balanced_accuracy_score,f1_score,precision_score
print(confusion_matrix(y_test,predict))
print(classification_report(y_test,predict))
print(balanced_accuracy_score(y_test,predict))